In [1]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 3.7 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import os
import openpyxl

from openpyxl import Workbook
from openpyxl.worksheet.filters import (FilterColumn, CustomFilter, CustomFilters, DateGroupItem, Filters)
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles import NamedStyle, Font, Border, Side, Alignment
from openpyxl.utils.cell import get_column_letter

In [3]:
print(os.getcwd())

/workspaces/Product-inventory-analytics


# Задаем функции для дальнейшего использования

In [4]:
def create_table(File):                 # Фукнция для создания таблицы
    itog.to_excel (File)

    wb = openpyxl.load_workbook(File)   # Исправляем наименование первого листа
    ws = wb.active
    ws.title  = "Итог"

    wb.save(File)

def groups(table, File):                # Функция для подсчета групп и добавления второго листа "Расшифровка"

    all = {}
    # Инфа по Группе 1 + (> 5 т.р.) + (> 10 т.р.)
    all['Count_1'] = table.query('Приход_Кол == 0 and Расход_кол == 0')['Name'].count()
    all['Sum_1'] = table.query('Приход_Кол == 0 and Расход_кол == 0')['Кон_ост_сумм'].sum()

    all['Count_1_5'] = table.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 5000')['Name'].count()
    all['Sum_1_5'] = table.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 5000')['Кон_ост_сумм'].sum()

    all['Count_1_10'] = table.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 10000')['Name'].count()
    all['Sum_1_10'] = table.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 10000')['Кон_ост_сумм'].sum()

    # Инфа по группе 2
    all['Count_2'] = table.query('Нач_ост_сумм > 0 and Приход_Сумм > 0 and Расход_сумм == 0')['Name'].count()
    all['Sum_2'] = table.query('Нач_ост_сумм > 0 and Приход_Сумм > 0 and Расход_сумм == 0')['Кон_ост_сумм'].sum()

    # Инфа по группе 3
    all['Count_3'] = table.query('Нач_ост_сумм > 0 and Расход_сумм > 0 and Кон_ост_кол > Расход_кол')['Name'].count()
    all['Sum_3'] = table.query('Нач_ост_сумм > 0 and Расход_сумм > 0 and Кон_ост_кол > Расход_кол')['Кон_ост_сумм'].sum()

    # Инфа по группе 4
    all['Count_4'] = table.query('Нач_ост_сумм == 0 and  Приход_Сумм > 0 and Расход_сумм == 0')['Name'].count()
    all['Sum_4'] = table.query('Нач_ост_сумм == 0 and  Приход_Сумм > 0 and Расход_сумм == 0')['Кон_ост_сумм'].sum()

    # Инфа по группе 5
    all['Count_5'] = table.query('Расход_кол >=3 and Кон_ост_кол == 0')['Name'].count()
    all['Sum_5'] = table.query('Расход_кол >=3 and Кон_ост_кол == 0')['Расход_сумм'].sum()
    
    decoding = pd.DataFrame({'Наименование': ['группа 1- "Мертвый" склад', 'группа 1 - свыше 5 тыс', 'группа 1 - свыше 10 тыс', 
                                              'группа 2 - Бестолковые закупки', 'группа 3 - Излишние запасы', 'группа 4 - Проверка спроса', 'группа 5 - Нет ходового товара'], 
                    'Сколько позиций.': [all['Count_1'], all['Count_1_5'], all['Count_1_10'], all['Count_2'], all['Count_3'], all['Count_4'], all['Count_5']], 
                    'Сумма.': [all['Sum_1'], all['Sum_1_5'], all['Sum_1_10'], all['Sum_2'], all['Sum_3'], all['Sum_4'], all['Sum_5']]})
    decoding = decoding.set_index('Наименование').round(2)  

    with pd.ExcelWriter(File, mode='a', if_sheet_exists= 'replace') as writer:      # Вносим информацию в лист
        decoding.to_excel(writer, sheet_name='Расшифровка',index = True)

def add_groups(table, File):                                                        # Функция для созданяия инфы по группам и внесения в excel листов по группам

    Group_1 = table.query('Приход_Кол == 0 and Расход_кол == 0')
    Group_1_5 = table.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 5000')
    Group_1_10 = table.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 10000')

    Group_2 = table.query('Нач_ост_сумм > 0 and Приход_Сумм > 0 and Расход_сумм == 0')
    Group_3 = table.query('Нач_ост_сумм > 0 and Расход_сумм > 0 and Кон_ост_сумм > Расход_сумм')
    Group_4 = table.query('Нач_ост_сумм == 0 and  Приход_Сумм > 0 and Расход_сумм == 0')
    Group_5 = table.query('Расход_кол >=3 and Кон_ост_кол == 0')

    wb = openpyxl.load_workbook(File)

    with pd.ExcelWriter(File, mode='a', if_sheet_exists= 'replace') as writer:  
        Group_1.to_excel(writer, sheet_name='Группа 1',index = False)
        Group_1_5.to_excel(writer, sheet_name='Группа 1 (>5)',index = False)
        Group_1_10.to_excel(writer, sheet_name='Группа 1 (>10)',index = False)
        Group_2.to_excel(writer, sheet_name='Группа 2',index = False)
        Group_3.to_excel(writer, sheet_name='Группа 3',index = False)
        Group_4.to_excel(writer, sheet_name='Группа 4',index = False)
        Group_5.to_excel(writer, sheet_name='Группа 5',index = False) 

In [5]:
# Оформление таблиц
def decor_file(File):                                             # Функция для  добавления всех таблиц в Exel

    wb = openpyxl.load_workbook(File) 

    for sheet in wb:                                              # для всех листов изменим ширину первого столбца
        if sheet.title in ['Итог', 'Расшифровка']:
            sheet.column_dimensions["A"].width = 30
        else:
            sheet.column_dimensions["A"].width = 60 
        for i in range(2, 10):                                    # для всех листов изменим ширину столбцов 2-9 
            letter = get_column_letter(i)                         # преобразовываем индекс столбца в его букву
            sheet.column_dimensions[letter].width = 18            # получаем ширину столбца
            sheet.freeze_panes = 'A2'

    ws = wb["Итог"]
    ws.column_dimensions["A"].width = 33
    for cell in ws["C2:C5"]:
        cell[0].style = 'Comma'
        cell[0].font = Font(size=11)
        cell[0].alignment = Alignment(horizontal='right')

    ws = wb["Расшифровка"]
    ws.column_dimensions["A"].width = 33
    for cell in ws["C2:C8"]:
        cell[0].style = 'Comma'
        cell[0].font = Font(size=11)
        cell[0].alignment = Alignment(horizontal='right')
    wb.save(File)

In [7]:
Oststki = pd.read_excel('Остатки.xlsx', header=10)

# Заменили пустые значения нулями
Oststki = Oststki.fillna(0)

# Убираем ненужные столбцы
# Oststki.drop(columns = ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Комментарии', 'остаток > расхода', 'стоимость за 1 ед'], axis=1, inplace = True)
Oststki.drop(columns = ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace = True)

Oststki.rename(columns = {'Номенклатура, № по кат., Производитель':'Name', 
                          'Количество (в ед. хранения)':'Нач_ост_кол',
                          'Сумма (в рег. валюте)':'Нач_ост_сумм',
                          'Количество (в ед. хранения).1':'Приход_Кол',
                          'Сумма (в рег. валюте).1':'Приход_Сумм',
                          'Количество (в ед. хранения).2':'Расход_кол',
                          'Сумма (в рег. валюте).2':'Расход_сумм',
                          'Количество (в ед. хранения).3':'Кон_ост_кол',
                          'Сумма (в рег. валюте).3':'Кон_ост_сумм',
                          }, inplace = True )

# Угданский ЧАТЦ

In [8]:
# Отберем данные для отдельной таблицы по Угданскому Чатц
start = Oststki.index [Oststki['Name']== "Угданский ЧАТЦ  склад " ]. tolist ()[0]
end = Oststki.index [Oststki['Name']== "Основной ЧАТЦ" ]. tolist ()[0]

# Функция для отбора только данным по Угданский Чатц
list = [i for i in range(start, end+1)]
Ugdan = Oststki.take(list, axis = 0)

## Данные (масла)

In [9]:
# Отберем данные для отдельной таблицы по Угданскому Чатц по маслам
start_oil = Ugdan.index [Ugdan['Name']== "Угданский ЧАТЦ  склад " ]. tolist ()[0]
end_oil = Ugdan.index [Ugdan['Name']== "Материалы, , " ]. tolist ()[0]

list_oil = [i for i in range(start_oil - start, end_oil-start)]

Ugdan_oil = Ugdan.take(list_oil, axis = 0)

# Исключим из таблицы строки, где Name включает в себя ', ,'
rule = [", ,"]
Ugdan_oil = Ugdan_oil[~Ugdan_oil.Name.str.contains('|'.join(rule))]

# Уберем ненужные строки
Ugdan_oil = Ugdan_oil.query("Name not in ['Угданский ЧАТЦ  склад ']")
# Подсчитаем нужные метрики
Quanity_oil = Ugdan_oil['Кон_ост_кол'].sum()
Sum_oil = Ugdan_oil['Кон_ост_сумм'].sum()
Sales_oil = Ugdan_oil['Расход_кол'].sum()
Sales_sum_oil = Ugdan_oil['Расход_сумм'].sum()
Sum_begin_oil = Ugdan_oil['Нач_ост_сумм'].sum()

## Данные (товары и запчасти)

In [10]:
start_prod = Ugdan.index [Ugdan['Name']== "Материалы, , " ]. tolist ()[0]
end_prod = Ugdan.index [Ugdan['Name']== "Основной ЧАТЦ" ]. tolist ()[0]

list_prod = [i for i in range(start_prod-start, end_prod-start)]

Ugdan_prod = Ugdan.take(list_prod, axis = 0)

# Исключим из таблицы строки, где Name включает в себя ', ,'
rule = [", ,"]
Ugdan_prod = Ugdan_prod[~Ugdan_prod.Name.str.contains('|'.join(rule))]
# Подсчитаем нужные метрики
Quanity_prod = Ugdan_prod['Кон_ост_кол'].sum()
Sum_prod = Ugdan_prod['Кон_ост_сумм'].sum()
Sales_prod = Ugdan_prod['Расход_кол'].sum()
Sales_sum_prod = Ugdan_prod['Расход_сумм'].sum()
Sum_begin_prod = Ugdan_prod['Нач_ост_сумм'].sum()

## Итоговые значения по первой таблице

In [11]:
itog_UGD = pd.DataFrame({'Склад': ['Угданский - масло', 'Угданский - запасные части', 'Итого', 'Продажи за ГОД в ценах с/с', 'Коэфф. оборачиваемости товара'], 
                    'Шт.': [Quanity_oil, Quanity_prod, Quanity_oil+Quanity_prod, Sales_oil+Sales_prod, '0'], 
                    'Сумма.': [Sum_oil, Sum_prod, Sum_oil+Sum_prod, Sales_sum_oil+Sales_sum_prod, (Sales_sum_oil+Sales_sum_prod)/((Sum_begin_oil+Sum_begin_prod+Sum_prod+Sum_oil)/2)]})
itog_UGD = itog_UGD.set_index('Склад').round(2)

itog_UGD

,Шт.,Сумма.
Склад,,
Угданский - масло,377.0,121688.08
Угданский - запасные части,5360.0,4762409.98
Итого,5737.0,4884098.06
Продажи за ГОД в ценах с/с,6791.5,4437375.99
Коэфф. оборачиваемости товара,0,0.88


In [12]:
# Исключим из таблицы строки, где Name включает в себя ', ,'
rule = [", ,"]
rule_2 = ["ЧАТЦ"]
Ugdan = Ugdan[~Ugdan.Name.str.contains('|'.join(rule))]
Ugdan = Ugdan[~Ugdan.Name.str.contains('|'.join(rule_2))]

## Выполнение функций для подсчета, внесения инфы и оформления

In [13]:
itog = itog_UGD
table = Ugdan
File = ('Угдан_апрель.xlsx')
create_table(File)
groups(table, File)
add_groups(table, File)
decor_file(File)

# МБ ТРАК

In [14]:
# Отберем данные для отдельной таблицы по МБ Трак
start_MB = Oststki.index [Oststki['Name']== "Основной склад (МБ ТРАК)" ]. tolist ()[0]
end_MB = Oststki.index [Oststki['Name']== "Склад материалов (МБ ТРАК)" ]. tolist ()[0]

list = [i for i in range(start_MB, end_MB+1)]                                # Функция для отбора только данным по Угданский Чатц
MB = Oststki.take(list, axis = 0)

MB.reset_index(drop= True , inplace= True )                                  # сразу обнулим индексы для последующей работы с данными

In [15]:
start_MB_masla = MB.index [MB['Name']== "Основной склад (МБ ТРАК)" ]. tolist ()[0]                 # Отберем данные для отдельной таблицы по МБ Трак (масла)
end_MB_masla = MB.index [MB['Name']== "Материалы, , " ]. tolist ()[0]

list = [i for i in range(start_MB_masla, end_MB_masla+1)]                                          # Функция для отбора только данных по МБ Трак (масла)
MB_masla = MB.take(list, axis = 0)

start_MB_tov = MB.index [MB['Name']== "Товары, , " ]. tolist ()[0]                                 # Отберем данные для отдельной таблицы по МБ Трак (товары)
end_MB_tov = MB.index [MB['Name']== "Склад материалов (МБ ТРАК)" ]. tolist ()[0]

list = [i for i in range(start_MB_tov, end_MB_tov+1)]                                              # Функция для отбора только данных по МБ Трак (товары)
MB_tov = MB.take(list, axis = 0)

In [16]:
# Исключим из таблицы строки, где Name включает в себя ', ,'
rule = [", ,"]
rule_2 = ["ТРАК"]

MB_masla = MB_masla[~MB_masla.Name.str.contains('|'.join(rule))]
MB_masla = MB_masla[~MB_masla.Name.str.contains('|'.join(rule_2))]

MB_tov = MB_tov[~MB_tov.Name.str.contains('|'.join(rule))]
MB_tov = MB_tov[~MB_tov.Name.str.contains('|'.join(rule_2))]

In [17]:
 # Подсчет всех значений для первой таблицы
Quanity_masla = MB_masla['Кон_ост_кол'].sum()
Sum_masla = MB_masla['Кон_ост_сумм'].sum()
Sales_masla = MB_masla['Расход_кол'].sum()
Sales_sum_masla = MB_masla['Расход_сумм'].sum()
Sum_begin_masla = MB_masla['Нач_ост_сумм'].sum()

Quanity_tov = MB_tov['Кон_ост_кол'].sum()
Sum_tov = MB_tov['Кон_ост_сумм'].sum()
Sales_tov = MB_tov['Расход_кол'].sum()
Sales_sum_tov = MB_tov['Расход_сумм'].sum()
Sum_begin_tov = MB_tov['Нач_ост_сумм'].sum()

In [18]:
itog_Trac = pd.DataFrame({'Склад': ['Масла и химия', 'Товары', 'Итого', 'Продажи за ГОД в ценах с/с', 'Коэфф. оборачиваемости товара'], 
                    'Шт.': [Quanity_masla, Quanity_tov, Quanity_masla+Quanity_tov, Sales_masla + Sales_tov, '0'], 
                    'Сумма.': [Sum_masla, Sum_tov, Sum_masla+Sum_tov, Sales_sum_masla+Sales_sum_tov, (Sales_sum_masla+Sales_sum_tov)/((Sum_begin_masla+Sum_begin_tov+Sum_masla+Sum_tov)/2)]})
itog_Trac = itog_Trac.set_index('Склад').round(2)

itog_Trac

,Шт.,Сумма.
Склад,,
Масла и химия,810.525,397059.76
Товары,909.0,13762197.94
Итого,1719.525,14159257.70
Продажи за ГОД в ценах с/с,3677.5,5286491.61
Коэфф. оборачиваемости товара,0,0.53


In [19]:
Trac = pd.concat([MB_masla, MB_tov]) # объединяем таблицы для выбора групп

# Выполнение функций для подсчета, внесения инфы и оформления

In [20]:
itog = itog_Trac
table = Trac
File = ('МБ_Трак_апрель.xlsx')
create_table(File)
groups(table, File)
add_groups(table, File)
decor_file(File)

# ТАХОГРАФЫ

In [22]:
# Отберем данные для отдельной таблицы по Тахографам
start_tah = Oststki.index [Oststki['Name']== "ТАХОГРАФЫ" ]. tolist ()[0]
end_tah = Oststki.index [Oststki['Name']== "Итого" ]. tolist ()[0]

# Функция для отбора только данных по Тахографам
list = [i for i in range(start_tah, end_tah+1)]
Tah = Oststki.take(list, axis = 0)

# сразу обнулим индексы для последующей работы с данными
Tah.reset_index(drop= True , inplace= True )

In [23]:
# Отберем данные для отдельной таблицы по товарам
start_tah_tov = Tah.index [Tah['Name']== "Товары, , " ]. tolist ()[0]
end_tah_tov = Tah.index [Tah['Name']== "Итого" ]. tolist ()[0]

# Функция для отбора только данных по товарам
list = [i for i in range(start_tah_tov, end_tah_tov+1)]
Tah_tov = Tah.take(list, axis = 0)

# Исключим из таблицы строки, где Name включает в себя ', ,'
rule = [", ,"]
rule_2 = ["Итого"]

Tah_tov = Tah_tov[~Tah_tov.Name.str.contains('|'.join(rule))]
Tah_tov = Tah_tov[~Tah_tov.Name.str.contains('|'.join(rule_2))]

In [24]:
# Подсчет всех значений
Quanity_Tah = Tah_tov['Кон_ост_кол'].sum()
Sum_Tah = Tah_tov['Кон_ост_сумм'].sum()
Sales_Tah = Tah_tov['Расход_кол'].sum()
Sales_sum_Tah = Tah_tov['Расход_сумм'].sum()
Sum_begin_Tah = Tah_tov['Нач_ост_сумм'].sum()

In [25]:
itog_Tah = pd.DataFrame({'Склад': ['Товары', 'Итого', 'Продажи за ГОД в ценах с/с', 'Коэфф. оборачиваемости товара'], 
                    'Шт.': [Quanity_Tah, Quanity_Tah, Sales_Tah, '0'], 
                    'Сумма.': [Sum_Tah, Sum_Tah, Sales_sum_Tah, (Sales_sum_Tah)/((Sum_begin_Tah+Sum_Tah)/2)]})
itog_Tah = itog_Tah.set_index('Склад').round(2)

itog_Tah

,Шт.,Сумма.
Склад,,
Товары,165.0,314184.97
Итого,165.0,314184.97
Продажи за ГОД в ценах с/с,14.0,80047.01
Коэфф. оборачиваемости товара,0,0.29


# Выполнение функций для подсчета, внесения инфы и оформления

In [26]:
itog=itog_Tah
table = Tah_tov
File = ('Тахографы_апрель.xlsx')
create_table(File)
groups(table, File)
add_groups(table, File)
decor_file(File)

# ГЕОДЕЗ

In [27]:
# Отберем данные для по Геодезу
start_geo = Oststki.index [Oststki['Name']== "Геодезическая ЧАТЦ склад" ]. tolist ()[0]
end_geo = Oststki.index [Oststki['Name']== "МОЛ Глазков Артем Дмитриевич" ]. tolist ()[0]

# Функция для отбора всех данных по Геодезу
list = [i for i in range(start_geo, end_geo+1)]
Geo = Oststki.take(list, axis = 0)

# сразу обнулим индексы для последующей работы с данными
Geo.reset_index(drop= True , inplace= True )

In [28]:
#Данные по материалам и товарам

# Отберем данные для по Геодезу (матариалы и товары)
start_geo_1 = Geo.index [Geo['Name']== "Материалы, , " ]. tolist ()[0]
end_geo_1 = Geo.index [Geo['Name']== "МОЛ Глазков Артем Дмитриевич" ]. tolist ()[0]

# Функция для отбора только данных по (матариалы и товары)
list = [i for i in range(start_geo_1, end_geo_1+1)]
Geo_1 = Geo.take(list, axis = 0)

# Исключим из таблицы строки, где Name включает в себя ', ,'
rule = [", ,"]
rule_2 = ["МОЛ"]

Geo_1 = Geo_1[~Geo_1.Name.str.contains('|'.join(rule))]

In [29]:
# Данные по товарам ЧТЦ сервис

# Отберем данные для по ООО ЧАТЦ
start_чатц = Oststki.index [Oststki['Name']== "ООО Читаавтотехцентр (сервис)" ]. tolist ()[0]
end_чатц = Oststki.index[Oststki['Name'].str.contains(r'\bУгданский')]. tolist ()[0]

# Функция для отбора всех данных по Геодезу
list = [i for i in range(start_чатц, end_чатц+1)]
ЧАТЦ = Oststki.take(list, axis = 0)

# сразу обнулим индексы для последующей работы с данными
ЧАТЦ.reset_index(drop= True , inplace= True )

In [30]:
# Отберем данные для по Основной склад компании
start_чатц = ЧАТЦ.index [ЧАТЦ['Name']== "Основной склад компании" ]. tolist ()[0]
end_чатц = ЧАТЦ.index[ЧАТЦ['Name'].str.contains(r'\bУгданский')]. tolist ()[0]

# Функция для отбора всех данных по Геодезу
list = [i for i in range(start_чатц, end_чатц+1)]
ЧАТЦ = ЧАТЦ.take(list, axis = 0)

# сразу обнулим индексы для последующей работы с данными
ЧАТЦ.reset_index(drop= True , inplace= True )

In [31]:
# Отберем данные для по товарам в Основной склад компании
start_чатц = ЧАТЦ.index [ЧАТЦ['Name']== "Товары, , " ]. tolist ()[0]
end_чатц = ЧАТЦ.index[ЧАТЦ['Name'].str.contains(r'\bУгданский')]. tolist ()[0]

# Функция для отбора всех данных по Геодезу
list = [i for i in range(start_чатц, end_чатц+1)]
ЧАТЦ = ЧАТЦ.take(list, axis = 0)

# сразу обнулим индексы для последующей работы с данными
ЧАТЦ.reset_index(drop= True , inplace= True )

# Исключим из таблицы строки, где Name включает в себя ', ,'
rule = [", ,"]
rule_2 = ["ЧАТЦ"]

ЧАТЦ = ЧАТЦ[~ЧАТЦ.Name.str.contains('|'.join(rule))]
ЧАТЦ = ЧАТЦ[~ЧАТЦ.Name.str.contains('|'.join(rule_2))]

## Подсчет значений для первой таблицы

In [32]:
# Подсчет всех значений по Геодез материалы и товары
Quanity_Geo_1 = Geo_1['Кон_ост_кол'].sum()
Sum_Geo_1 = Geo_1['Кон_ост_сумм'].sum()
Sales_Geo_1 = Geo_1['Расход_кол'].sum()
Sales_sum_Geo_1 = Geo_1['Расход_сумм'].sum()
Sum_begin_Geo_1 = Geo_1['Нач_ост_сумм'].sum()

# Подсчет всех значений по ЧАТЦ товары
Quanity_ЧАТЦ = ЧАТЦ['Кон_ост_кол'].sum()
Sum_ЧАТЦ = ЧАТЦ['Кон_ост_сумм'].sum()
Sales_ЧАТЦ = ЧАТЦ['Расход_кол'].sum()
Sales_sum_ЧАТЦ = ЧАТЦ['Расход_сумм'].sum()
Sum_begin_ЧАТЦ = ЧАТЦ['Нач_ост_сумм'].sum()

In [33]:
itog_Geo = pd.DataFrame({'Склад': ['Материалы и тов. Геодез', 'Товары ЧТЦ', 'Итого', 'Продажи за ГОД в ценах с/с', 'Коэфф. оборачиваемости товара'], 
                    'Шт.': [Quanity_Geo_1,  Quanity_ЧАТЦ, Quanity_Geo_1+Quanity_ЧАТЦ, Sales_Geo_1+Sales_ЧАТЦ, '0'], 
                    'Сумма.': [Sum_Geo_1, Sum_ЧАТЦ, Sum_Geo_1+Sum_ЧАТЦ, Sales_sum_Geo_1+Sales_sum_ЧАТЦ, (Sales_sum_Geo_1+Sales_sum_ЧАТЦ)/((Sum_begin_Geo_1+Sum_begin_ЧАТЦ+Sum_Geo_1+Sum_ЧАТЦ)/2)]})
itog_Geo = itog_Geo.set_index('Склад').round(2)

itog_Geo

,Шт.,Сумма.
Склад,,
Материалы и тов. Геодез,10388.5,27237672.00
Товары ЧТЦ,16.0,59469.17
Итого,10404.5,27297141.17
Продажи за ГОД в ценах с/с,14399.5,35591678.03
Коэфф. оборачиваемости товара,0,1.42


In [34]:
All = pd.concat([Geo_1, ЧАТЦ]) # объединяем таблицы для выбора групп

# Выполнение функций для подсчета, внесения инфы и оформления

In [35]:
itog=itog_Geo
table = All
File = ('Геодез_и_ЧТЦ_апрель.xlsx')
create_table(File)
groups(table, File)
add_groups(table, File)
decor_file(File)

# МАСЛА

## Масла Геодез магазин

In [36]:
# Отберем данные по Маслам в Геодез магазине
start_oil_1 = Oststki.index [Oststki['Name']== "Геодезическая ЧАТЦ склад" ]. tolist ()[0]
end_oil_1 = Oststki.index [Oststki['Name']== "Материалы, , " ]. tolist ()[0]

# Функция для отбора всех данных по Геодезу
list = [i for i in range(start_oil_1, end_oil_1+1)]
OIL_1 = Oststki.take(list, axis = 0)

# Исключим из таблицы строки, где Name включает в себя ', ,'
rule = [", ,"]
rule_2 = ["ЧАТЦ"]

OIL_1 = OIL_1[~OIL_1.Name.str.contains('|'.join(rule))]
OIL_1 = OIL_1[~OIL_1.Name.str.contains('|'.join(rule_2))]

## Масла ЧТЦ

In [37]:
# Отберем данные по Маслам в ЧТЦ сервисе
start_oil_2 = Oststki.index [Oststki['Name']== "ООО Читаавтотехцентр (сервис)" ]. tolist ()[0]
end_oil_2 = Oststki.index[Oststki['Name'].str.contains(r'\bУгданский')]. tolist ()[0]

# Функция для отбора данных
list = [i for i in range(start_oil_2, end_oil_2+1)]
OIL_2 = Oststki.take(list, axis = 0)

# сразу обнулим индексы для последующей работы с данными
OIL_2.reset_index(drop= True , inplace= True )

In [38]:
# Отберем данные для по Основной склад компании
start_oil_2 = OIL_2.index [OIL_2['Name']== "Основной склад компании" ]. tolist ()[0]
end_oil_2 = OIL_2.index[OIL_2['Name'].str.contains(r'\bУгданский')]. tolist ()[0]

# Функция для отбора всех данных по Геодезу
list = [i for i in range(start_oil_2, end_oil_2+1)]
OIL_2 = OIL_2.take(list, axis = 0)

# сразу обнулим индексы для последующей работы с данными
OIL_2.reset_index(drop= True , inplace= True )

In [39]:
# Отберем данные для по маслам
start_oil_2 = OIL_2.index [OIL_2['Name']== "Основной склад компании" ]. tolist ()[0]
end_oil_2 = OIL_2.index [OIL_2['Name']== "Материалы, , " ]. tolist ()[0]

# Функция для отбора
list = [i for i in range(start_oil_2, end_oil_2+1)]
OIL_2 = OIL_2.take(list, axis = 0)

# Исключим из таблицы строки, где Name включает в себя ', ,'
rule = [", ,"]
rule_2 = ["склад"]

OIL_2 = OIL_2[~OIL_2.Name.str.contains('|'.join(rule))]
OIL_2 = OIL_2[~OIL_2.Name.str.contains('|'.join(rule_2))]

## Подсчет значений для первой таблицы

In [40]:
# Подсчет всех значений по маслам Геодез
Quanity_1 = OIL_1['Кон_ост_кол'].sum()
Sum_1 = OIL_1['Кон_ост_сумм'].sum()
Sales_1 = OIL_1['Расход_кол'].sum()
Sales_sum_1 = OIL_1['Расход_сумм'].sum()
Sum_begin_1 = OIL_1['Нач_ост_сумм'].sum()

# Подсчет всех значений по маслам ЧАТЦ
Quanity_2 = OIL_2['Кон_ост_кол'].sum()
Sum_2 = OIL_2['Кон_ост_сумм'].sum()
Sales_2 = OIL_2['Расход_кол'].sum()
Sales_sum_2 = OIL_2['Расход_сумм'].sum()
Sum_begin_2 = OIL_2['Нач_ост_сумм'].sum()

In [41]:
itog_oil = pd.DataFrame({'Склад': ['Масла Геодез', 'Масла ЧАТЦ', 'Итого', 'Продажи за ГОД в ценах с/с', 'Коэфф. оборачиваемости товара'], 
                    'Шт.': [Quanity_1,  Quanity_2, Quanity_1+Quanity_2, Sales_1+Sales_2, '0'], 
                    'Сумма.': [Sum_1, Sum_2, Sum_1+Sum_2, Sales_sum_1+Sales_sum_2, (Sales_sum_1+Sales_sum_2)/((Sum_begin_1+Sum_begin_2+Sum_1+Sum_2)/2)]})
itog_oil = itog_oil.set_index('Склад').round(2)

itog_oil

,Шт.,Сумма.
Склад,,
Масла Геодез,2561.25,1131895.11
Масла ЧАТЦ,0.0,0.00
Итого,2561.25,1131895.11
Продажи за ГОД в ценах с/с,8426.15,2869879.65
Коэфф. оборачиваемости товара,0,2.39


In [42]:
All_oil = pd.concat([OIL_1, OIL_2]) # объединяем таблицы для выбора групп

## Выполнение функций для подсчета, внесения инфы и оформления

In [43]:
itog=itog_oil
table = All_oil
File = ('Масла_апрель.xlsx')
create_table(File)
groups(table, File)
add_groups(table, File)
decor_file(File)